# 코로나 네이버 뉴스 API 감성분석
    머신러닝2_6일차 참조

# 네이버 뉴스 크롤링

In [3]:
import pandas as pd
import numpy as np

In [4]:
# 함수 설계
import os
import sys
import urllib.request
import json
client_id = "J2KwXxSxDGbKmuP9V6eE"
client_secret = "NOtfPphkJY"
def getRequestUrl(url):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    try:
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200): # 성공
            response_body = response.read()
            return response_body.decode('utf-8')
        else:
            print("Error Code:" + rescode)  # 404(경로가 잘못), 500(내부에러)
            return None
    except Exception as e:
        print(e)
        return None
        
def getNeverSearch(srcText,searchType='news',display=10,start=1):
    encText = urllib.parse.quote(srcText)
    url = f"https://openapi.naver.com/v1/search/{searchType}.json?query={encText}&display={display}&start={start}&sort=sim"
    jsonData =  getRequestUrl(url)
    if jsonData == None:
        return None
    else:
        return json.loads(jsonData)

In [5]:
searchRange = [i for i in range(1,1000,100)] + [1000]  
result = []
for start in searchRange:
    result += getNeverSearch('코로나',start=start, display=100)['items']

In [17]:
### 분석에 필요한 데이터 추출    
data_title = []
data_description = []
for item in result:
    data_title.append(item['title'])
    data_description.append(item['description'])

In [23]:
data_df = pd.DataFrame({'title':data_title, 'description' : data_description})

In [24]:
data_df.head()

,title,description
0,"“<b>코로나</b> 백신, 고위험군 年2회 접종을”",지영미 질병관리청장이 앞으로 “신종 <b>코로나</b>바이러스 감염증(<b>코로나<...
1,"방역당국 &quot;<b>코로나</b>백신 정례접종, 변이예측·백신개발 보며 검토&...","방역 당국이 <b>코로나</b>19 백신의 정례접종 도입을 검토할 때 변이 예측, ..."
2,방역당국 &quot;<b>코로나</b> 백신 연 1~2회 정기접종 검토 중&quot;,방역당국이 <b>코로나</b>19 변이 예측과 백신 개발 상황 등을 고려해 백신 정...
3,폐업신고 않고 매출실적 낮춰 <b>코로나</b> 지원금 ‘꿀꺽’,특히 <b>코로나</b>19 소상공인 지원금 부정수급 신고 관련 상담이 1년 전보다...
4,"&apos;<b>코로나</b> 수혜&apos; 줌, 직원 1300명 감원 발표",<b>코로나</b>19 팬데믹 대표 수혜 기업 중 하나인 화상회의 업체 줌 비디오 ...


# title에 대한 감성 분석
    1.감성분석(td-idf)을 실시할 데이터의 피처 벡터화
    2.감성분석 - 로지스틱회귀 - 파라메터는 하이터 튜닝
    3.결과를 프레임에 저장

In [28]:
from konlpy.tag import Okt
okt = Okt()
# 토큰화 작업을 수행할 함수를 작성
def okt_tokenizer(text):
    tokens =  okt.morphs(text)
    return tokens
# tf-idf로 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer
# 모델
tfidf =  TfidfVectorizer(tokenizer=okt_tokenizer,
                ngram_range=(1,2),         #  토큰의 단어크기를 1~2개 단어로
                min_df=3,                  # 출현 빈도가 최소 3번 이상
                max_df= 0.9                # 최대 90% 이하인 것만 사용
               )
tfidf.fit(data_df['title'])

C:\Users\user\miniconda3\envs\py\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                tokenizer=<function okt_tokenizer at 0x000001E57ACFB760>)

In [30]:
# 1
data_title_tfidf = tfidf.transform(data_df['title'])
data_title_tfidf.shape

(1100, 1626)

In [37]:
# 이전에서 학습했던 학습용 데이터를 가지고 머신러닝 모델을 만든다
train_df = pd.read_csv('ratings_train.txt',encoding='utf-8',sep='\t')
train_df.dropna(subset=['document'],axis = 0,inplace=True)
train_df.reset_index(drop=True,inplace=True)
import re
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r'[^ㄱ-ㅎ가-힣]+'," ",x))
train_tfidf = tfidf.transform(train_df['document'])

In [35]:
#2 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression(random_state=0)
param ={
    'C':range(1,10)
}
gsc = GridSearchCV(lr,param_grid=param, cv=3,scoring='accuracy',verbose = 1)
# 문장과 (학습)  긍정 부정(타겟)
gsc.fit(train_tfidf,train_df['label'])

In [33]:
help(GridSearchCV)

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "score_samples", "predict", "predict_proba",
 |  "decision_function", "transform" and "inverse_transform" if they are
 |  implemented in the estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object
 |      This is assumed to implement the scikit-learn estimator interface.
 |      Either est